# ArXiv 每日論文推薦檢視器

這個 Google Colab 筆記本會自動從 ArXiv 上爬取指定領域的最新論文，並根據您設定的關鍵字，使用 TF-IDF 和餘弦相似度計算論文的相關性分數，最後**直接在 Colab 中顯示每日論文推薦結果**。

### 流程概覽：
1.  **安裝與導入函式庫**：設定執行環境。
2.  **參數配置**：設定您的個人偏好，包括關鍵字和 ArXiv 領域。
3.  **抓取每日論文**：從 ArXiv API 獲取最新發表的論文。
4.  **匹配與排序**：計算每篇論文與關鍵字的相關性分數。
5.  **顯示推薦結果**：將最終推薦列表清晰地呈現在下方儲存格中。

## 步驟 1：安裝與導入函式庫

首先，我們需要安裝 `arxiv` 套件來與 ArXiv API 互動。其他如 `pandas` 和 `scikit-learn` 在 Colab 環境中已預先安裝。

In [ ]:
!pip install arxiv

import arxiv
import pandas as pd
from datetime import datetime, timedelta, timezone
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 步驟 2：參數配置

在這裡設定您的個人偏好。您可以修改 `KEYWORDS` 列表來定義您感興趣的研究方向，並在 `ARXIV_TOPIC` 中選擇您想關注的 ArXiv 領域。

In [ ]:
# --- 論文相關設定 ---
# 您感興趣的關鍵字
KEYWORDS = [
    'Object Detection', 'Computer Vision', 'Image Classification',
    'Natural Language Processing', 'Large Language Models', 'Transformer',
    'Reinforcement Learning', 'Graph Neural Networks', 'Recommendation Systems'
]

# 您想搜尋的 ArXiv 領域 (key 是自訂名稱, value 是 ArXiv 的官方代碼)
ARXIV_TOPIC = {
    'Computer Vision': 'cs.CV',
    'Computation and Language': 'cs.CL',
    'Artificial Intelligence': 'cs.AI',
    'Machine Learning': 'stat.ML'
}

MAX_RESULTS_PER_TOPIC = 50  # 每個領域最多爬取幾篇論文
TOP_K = 10                  # 最終顯示幾篇最相關的論文

print("✅ 參數設定完成！")
print(f"設定的關鍵字數量: {len(KEYWORDS)}")
print(f"將搜尋的領域: {list(ARXIV_TOPIC.keys())}")

## 步驟 3：抓取每日論文

這個函式會連接 ArXiv API，抓取指定領域中在過去 24 小時內發表的論文，並將標題、摘要、作者等資訊整理成一個 `pandas` DataFrame。

In [ ]:
import arxiv
import pandas as pd
from datetime import datetime, timedelta, timezone
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_daily_papers(topics, max_results=50):
    """
    從 ArXiv API 抓取指定領域的每日最新論文。
    """
    # 設定時區為 ArXiv API 使用的 UTC，並取得一天前的時間
    utc_now = datetime.now(timezone.utc)
    yesterday_utc = utc_now - timedelta(days=1)

    all_papers = []
    query = " OR ".join([f'cat:{cat}' for cat in topics.values()])

    search = arxiv.Search(
        query=query,
        max_results=max_results * len(topics), # 總論文數
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    print(f"🔍 正在從 ArXiv 搜尋論文... (查詢語法: {query})")
    for result in search.results():
        # ArXiv API 回傳的時間是 UTC，我們直接比較
        if result.published.date() >= yesterday_utc.date():
            paper_info = {
                'title': result.title,
                'summary': result.summary.replace('\n', ' '),
                'authors': [author.name for author in result.authors],
                'url': result.pdf_url,
                'published': result.published.strftime("%Y-%m-%d")
            }
            all_papers.append(paper_info)

    if not all_papers:
        print("⚠️ 找不到過去一天內發表的新論文。")
        return pd.DataFrame()

    df = pd.DataFrame(all_papers)
    df = df.drop_duplicates(subset='title', keep='first') # 移除重複標題
    print(f"✅ 成功抓取並整理了 {len(df)} 篇不重複的論文。")
    return df

# --- 執行抓取 ---
daily_papers_df = get_daily_papers(ARXIV_TOPIC, max_results=MAX_RESULTS_PER_TOPIC)
if not daily_papers_df.empty:
    print("\n--- 抓取結果預覽 ---")
    display(daily_papers_df.head())

## 步驟 4：匹配與排序

這個函式是推薦系統的核心。它執行以下操作：
1.  將您設定的關鍵字合併成一個「興趣描述」。
2.  使用 `TfidfVectorizer` 將每篇論文的摘要和您的興趣描述轉換成數值向量。
3.  計算每篇論文摘要向量與您的興趣描述向量之間的**餘弦相似度**，得到一個 0 到 1 之間的分數。
4.  根據分數排序，分數越高的論文代表與您的興趣越相關。

In [ ]:
def keyword_match_and_rank(df, keywords, top_k=10):
    """
    使用 TF-IDF 和餘弦相似度，根據關鍵字對論文進行排序。
    """
    if df.empty:
        print("資料為空，無法進行匹配。")
        return df

    print("\n⚖️ 正在計算論文與關鍵字的相關性分數...")
    # 將論文摘要和關鍵字列表合併，以便一同轉換為 TF-IDF 向量
    keyword_string = ' '.join(keywords)
    corpus = df['summary'].tolist() + [keyword_string]

    # 初始化並擬合 TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 計算論文摘要向量與關鍵字向量的餘弦相似度
    cosine_sim = cosine_similarity(tfidf_matrix[:-1], tfidf_matrix[-1])

    # 將相似度分數加入 DataFrame 並排序
    df['score'] = cosine_sim
    ranked_df = df.sort_values(by='score', ascending=False)

    print(f"✅ 計算完成！將選出前 {top_k} 篇最相關的論文。")
    return ranked_df.head(top_k)

# --- 執行匹配與排序 ---
recommended_papers_df = keyword_match_and_rank(daily_papers_df, KEYWORDS, top_k=TOP_K)
if not recommended_papers_df.empty:
    print("\n--- 推薦結果預覽 ---")
    display(recommended_papers_df[['title', 'score', 'published']])

## 步驟 5：整合執行與檢視結果

最後，我們將所有函式串連起來。您可以直接執行此儲存格來完成整個流程：從抓取論文到評分排序，並在下方直接檢視最終的推薦結果。

In [ ]:
def main():
    """主執行函式"""
    print("--- 開始執行每日論文推薦任務 ---")

    # 1. 抓取論文
    papers_df = get_daily_papers(
        topics=ARXIV_TOPIC,
        max_results=MAX_RESULTS_PER_TOPIC
    )

    # 2. 匹配與排序
    ranked_papers = keyword_match_and_rank(
        df=papers_df,
        keywords=KEYWORDS,
        top_k=TOP_K
    )

    # 3. 顯示最終推薦結果
    if not ranked_papers.empty:
        print("\n--- ✨ 最終為您推薦的論文列表 ✨ ---")
        # 為了更好的可讀性，我們讓 URL 可以點擊
        def make_clickable(val):
            return f'<a target="_blank" href="{val}">{val}</a>'

        display_df = ranked_papers[['title', 'score', 'published', 'url']].copy()
        display_df.style.format({'url': make_clickable})
        display(display_df)
    else:
        print("\n--- 今日無合適論文推薦 ---")

    print("\n--- 任務執行完畢 ---")

# --- 執行主函式 ---
if __name__ == "__main__":
    main()

## 自動化建議

這個 Colab 筆記本非常適合用於教學。如果您想實現真正的學術研究每日自動化，可以考慮以下方法：

1.  **轉換為 Python 腳本**：將此筆記本的程式碼整理成一個 `.py` 檔案。
2.  **使用 GitHub Actions**：您可以設定一個排程，讓 GitHub 每天在固定的時間自動執行您的 Python 腳本。如果未來需要加入其他 API 金鑰等秘密，可以使用 GitHub Repository 的 Secrets 功能。
3.  **使用雲端服務**：例如以 Google Cloud Scheduler 搭配 Cloud Functions，來設定定時任務。